Pubmed

TITLE ( "mental health" OR "mental-health" OR "well-being" OR "well being" OR wellbeing ) AND TITLE ( "built environment" OR "building architecture" OR "architectural design" OR "building design" OR "environmental design" OR "urban architecture" OR "urban environment" OR "sustainable architecture" ) AND NOT DOCTYPE ( re ) AND NOT DOCTYPE ( "ma" ) AND PUBYEAR > 2013 AND PUBYEAR < 2025 AND PUBYEAR > 2013 AND PUBYEAR < 2025 AND PUBYEAR > 2013 AND PUBYEAR < 2025 AND ( LIMIT-TO ( SRCTYPE , "j" ) OR LIMIT-TO ( SRCTYPE , "p" ) ) AND ( LIMIT-TO ( PUBSTAGE , "final" ) OR LIMIT-TO ( PUBSTAGE , "aip" ) ) AND ( LIMIT-TO ( SUBJAREA , "SOCI" ) OR LIMIT-TO ( SUBJAREA , "ENGI" ) OR LIMIT-TO ( SUBJAREA , "ENVI" ) OR LIMIT-TO ( SUBJAREA , "COMP" ) OR LIMIT-TO ( SUBJAREA , "ENER" ) OR LIMIT-TO ( SUBJAREA , "ARTS" ) OR LIMIT-TO ( SUBJAREA , "PSYC" ) OR LIMIT-TO ( SUBJAREA , "HEAL" ) OR LIMIT-TO ( SUBJAREA , "MULT" ) OR LIMIT-TO ( SUBJAREA , "MATE" ) OR LIMIT-TO ( SUBJAREA , "NEUR" ) OR LIMIT-TO ( SUBJAREA , "MEDI" ) ) AND ( LIMIT-TO ( DOCTYPE , "ar" ) OR LIMIT-TO ( DOCTYPE , "cp" ) ) AND ( LIMIT-TO ( LANGUAGE , "English" ) )

Ebsco

("mental health" OR "mental-health" OR "well-being" OR "well being" OR wellbeing) AND ("built environment" OR "building architecture" OR "architectural design" OR "building design" OR "environmental design" OR "urban architecture" OR "urban environment" OR "Sustainable architecture")

Scopus

( ( “mental health” OR “mental-health” OR “well-being” OR “well being” OR wellbeing ) ) AND ( ( “built environment” OR “building architecture” OR “architectural design” OR “building design” OR “environmental design” OR “urban architecture” OR “urban environment” OR "Sustainable architecture” ) ) NOT "review" NOT meta-analysis

Google Scholar 

allintitle: ("mental health" OR "well being" OR "wellbeing") AND ( "built environment" OR "building architecture" OR "architectural design" OR "building design" OR "environmental design" OR "urban architecture" OR "urban environment" OR "sustainable architecture") -review -"meta analysis"

In [ ]:
import os
import requests
#Elsevier API Key
API_KEY = "insert API KEY"

Base_URL = "https://api.elsevier.com/content/search/scopus"

In [2]:
#headers for the API request
HEADERS = {
    "X-ELS-APIKey": API_KEY,
    "Accept": "application/json"
}

def get_papers(query, start = 0, count = 25):
    """
    Fetch papers from Elsevier API based on the query.
    
    Parameters:
    - query: The search query string.
    - start: The starting index for pagination.
    - count: The number of results to return.
    
    Returns:
    - A list of papers matching the query.
    """
    params = {
        "query": query,
        "start": start,
        "count": count
    }
    response = requests.get(Base_URL, headers = HEADERS, params = params)
    
    #Raise an error for bad responses
    response.raise_for_status()
    
    data = response.json()
    total = int(data['search-results']['opensearch:totalResults'])
    items = data['search-results']['entry']
    return total, items

    

In [3]:
def filter_articles(articles, keyword=None, year_min=None):
    out = []
    for art in articles:
        title    = art.get("dc:title", "")
        abstract = art.get("dc:description", "")
        doi      = art.get("prism:doi", "")
        paper_url = f"https://doi.org/{doi}" if doi else ""

        text = (title + " " + abstract).lower()
        if keyword and keyword.lower() not in text:
            continue

        out.append({
            "title":     title,
            "year":      int(art.get("prism:coverDate","0000")[:4] or 0),
            "doi":       doi,
            "paper_url": paper_url,
        })
    return out


In [4]:
import math

def fetch_and_filter_all(query, page_size=25, keyword=None):
    total, _ = get_papers(query, start=0, count=1)
    page_count = math.ceil(total / page_size)
    print(f"Total matches: {total} → {page_count} pages")

    all_filtered = []
    for i, offset in enumerate(range(0, total, page_size), start=1):
        print(f"Fetching page {i}/{page_count}", end="\r", flush=True)
        _, page = get_papers(query, start=offset, count=page_size)
        #only pass the articles list and keyword
        filtered = filter_articles(page, keyword=keyword)
        all_filtered.extend(filtered)

    print()
    return all_filtered


In [5]:
my_query = (    'TITLE-AB("mental health" OR "mental-health" OR "well-being" OR "well being" OR wellbeing) AND TITLE-AB("built environment" OR "building architecture" OR "architectural design" OR "building design" OR "environmental design" OR "urban architecture" OR "urban environment" OR "sustainable architecture") AND NOT DOCTYPE(re) AND NOT DOCTYPE("ma") AND ( LIMIT-TO ( SRCTYPE,"j" ) OR LIMIT-TO ( SRCTYPE,"p" ) ) AND ( LIMIT-TO ( PUBSTAGE,"final" ) OR LIMIT-TO ( PUBSTAGE,"aip" ) ) AND ( LIMIT-TO ( DOCTYPE,"ar" ) OR LIMIT-TO ( DOCTYPE,"cp" ) )  AND ( LIMIT-TO ( LANGUAGE,"English" ) )')
    
#first n pages
total, page = get_papers(my_query, start=0, count=25)
print(f"Found {total} articles. Retrieving first {len(page)}…")
    
filtered = filter_articles(
    page,
    year_min=2014,
    keyword="neural"
)
    
#Display results
for art in filtered:
    print(f"{art['year']} – {art['title']} ({', '.join(art['authors'])})")
    print(f"  DOI: {art['doi']}")
    print(f"  URL: {art['url']}\n")

HTTPError: 400 Client Error: Bad Request for url: https://api.elsevier.com/content/search/scopus?query=TITLE-AB%28%22mental+health%22+OR+%22mental-health%22+OR+%22well-being%22+OR+%22well+being%22+OR+wellbeing%29+AND+TITLE-AB%28%22built+environment%22+OR+%22building+architecture%22+OR+%22architectural+design%22+OR+%22building+design%22+OR+%22environmental+design%22+OR+%22urban+architecture%22+OR+%22urban+environment%22+OR+%22sustainable+architecture%22%29+AND+NOT+DOCTYPE%28re%29+AND+NOT+DOCTYPE%28%22ma%22%29+AND+%28+LIMIT-TO+%28+SRCTYPE%2C%22j%22+%29+OR+LIMIT-TO+%28+SRCTYPE%2C%22p%22+%29+%29+AND+%28+LIMIT-TO+%28+PUBSTAGE%2C%22final%22+%29+OR+LIMIT-TO+%28+PUBSTAGE%2C%22aip%22+%29+%29+AND+%28+LIMIT-TO+%28+DOCTYPE%2C%22ar%22+%29+OR+LIMIT-TO+%28+DOCTYPE%2C%22cp%22+%29+%29++AND+%28+LIMIT-TO+%28+LANGUAGE%2C%22English%22+%29+%29&start=0&count=25

In [9]:
import csv

year_ranges = [(2014, 2015), (2016, 2017), (2018, 2019), (2020, 2021), (2022, 2023), (2024, 2025)]
all_results = []

for y0, y1 in year_ranges:
    slice_q = f'{my_query} AND PUBYEAR > {y0-1} AND PUBYEAR < {y1+1}'
    print(f"\nFetching {y0}–{y1} slice…")
    slice_results = fetch_and_filter_all(
        slice_q,
        page_size=25,
        keyword=None   # or your text‐filter here
    )
    all_results.extend(slice_results)


keys = ["title", "year", "doi", "paper_url"]
with open("filtered_papers.csv", "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=keys)
    writer.writeheader()
    writer.writerows(all_results)


print(f"\nDone! {len(all_results)} papers saved to filtered_papers.csv")



Fetching 2014–2015 slice…
Total matches: 1317 → 53 pages


KeyboardInterrupt: 

In [ ]:
query = ("TITLE-ABS-KEY("mental health" OR "mental-health" OR "well-being" OR "well being" OR wellbeing) AND KEY("built environment" OR "building architecture" OR "architectural design" OR "building design" OR "environmental design" OR "urban architecture" OR "urban environment" OR "sustainable architecture") AND NOT DOCTYPE(re) AND NOT DOCTYPE("ma") AND ( LIMIT-TO ( SRCTYPE,"j" ) OR LIMIT-TO ( SRCTYPE,"p" ) ) AND ( LIMIT-TO ( PUBSTAGE,"final" ) OR LIMIT-TO ( PUBSTAGE,"aip" ) ) AND ( LIMIT-TO ( DOCTYPE,"ar" ) OR LIMIT-TO ( DOCTYPE,"cp" ) ) AND ( LIMIT-TO ( PUBYEAR,2014) OR LIMIT-TO ( PUBYEAR,2015) OR LIMIT-TO ( PUBYEAR,2016) OR LIMIT-TO ( PUBYEAR,2017) OR LIMIT-TO ( PUBYEAR,2018) OR LIMIT-TO ( PUBYEAR,2019) OR LIMIT-TO ( PUBYEAR,2020) OR LIMIT-TO ( PUBYEAR,2021) OR LIMIT-TO ( PUBYEAR,2022) OR LIMIT-TO ( PUBYEAR,2023) OR LIMIT-TO ( PUBYEAR,2024) ) AND ( LIMIT-TO ( LANGUAGE,"English" ) )")
 

In [35]:
import pandas as pd

paper_df = pd.read_csv("filtered_papers.csv")

(    'TITLE("mental health" OR "mental-health" OR "well-being" OR "well being" OR wellbeing) AND TITLE("built environment" OR "building architecture" OR "architectural design" OR "building design" OR "environmental design" OR "urban architecture" OR "urban environment" OR "sustainable architecture") AND NOT DOCTYPE(re) AND NOT DOCTYPE("ma") AND PUBYEAR > 2013 AND PUBYEAR < 2025 AND PUBYEAR > 2013 AND PUBYEAR < 2025 AND PUBYEAR > 2013 AND PUBYEAR < 2025 AND ( LIMIT-TO ( SRCTYPE,"j" ) OR LIMIT-TO ( SRCTYPE,"p" ) ) AND ( LIMIT-TO ( PUBSTAGE,"final" ) OR LIMIT-TO ( PUBSTAGE,"aip" ) ) AND ( LIMIT-TO ( SUBJAREA,"SOCI" ) OR LIMIT-TO ( SUBJAREA,"ENGI" ) OR LIMIT-TO ( SUBJAREA,"ENVI" ) OR LIMIT-TO ( SUBJAREA,"COMP" ) OR LIMIT-TO ( SUBJAREA,"ENER" ) OR LIMIT-TO ( SUBJAREA,"ARTS" ) OR LIMIT-TO ( SUBJAREA,"PSYC" ) OR LIMIT-TO ( SUBJAREA,"HEAL" ) OR LIMIT-TO ( SUBJAREA,"MULT" ) OR LIMIT-TO ( SUBJAREA,"MATE" ) OR LIMIT-TO ( SUBJAREA,"NEUR" ) OR LIMIT-TO ( SUBJAREA,"MEDI" ) ) AND ( LIMIT-TO ( DOCTYPE,"ar" ) OR LIMIT-TO ( DOCTYPE,"cp" ) ) AND ( LIMIT-TO ( LANGUAGE,"English" ) )')
    

In [36]:
paper_df

,title,year,doi,paper_url
0,Perceived urban environment elements associate...,2025,10.1038/s41598-025-88349-x,https://doi.org/10.1038/s41598-025-88349-x
1,"Impact of windows, natural materials, and dive...",2025,10.1016/j.buildenv.2025.113147,https://doi.org/10.1016/j.buildenv.2025.113147
2,Nonlinear interactive associations between bui...,2025,10.1016/j.apgeog.2025.103662,https://doi.org/10.1016/j.apgeog.2025.103662
3,Urban environment perceptions and mental well-...,2025,10.1093/heapro/daaf066,https://doi.org/10.1093/heapro/daaf066
4,Mental health facility visits before and after...,2025,10.1016/j.jth.2025.102037,https://doi.org/10.1016/j.jth.2025.102037
...,...,...,...,...
931,Urban environment and mental health. A longitu...,1997,10.1192/bjp.171.6.530,https://doi.org/10.1192/bjp.171.6.530
932,The urban environment and mental health,1992,10.1177/002076409203800102,https://doi.org/10.1177/002076409203800102
933,The urban environment and mental health. Evalu...,1981,NaN,NaN
934,Identification of mental health problems in an...,1981,NaN,NaN
